<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/Schachcomputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
bauer = 1 # 1
springer = 2 # 3
laeufer = 3 # 3
turm = 4 # 5
dame = 5 # 9
koenig = 6 # 100
leer = 0
def reset_schachbrett():
    schachbrett = [
        [-turm, -springer, -laeufer, -dame, -koenig, -laeufer, -springer, -turm],
        [-bauer, -bauer, -bauer, -bauer, -bauer, -bauer, -bauer, -bauer],
        [leer, leer, leer, leer, leer, leer, leer, leer],
        [leer, leer, leer, leer, leer, leer, leer, leer],
        [leer, leer, leer, leer, leer, leer, leer, leer],
        [leer, leer, leer, leer, leer, leer, leer, leer],
        [bauer, bauer, bauer, bauer, bauer, bauer, bauer, bauer],
        [turm, springer, laeufer, dame, koenig, laeufer, springer, turm]
    ]
    return schachbrett
schachbrett = reset_schachbrett()

print(schachbrett)
figuren = ["\u2659", "\u2659","\u2658","\u2657","\u2656","\u2655","\u2654","\u265A","\u265B","\u265C","\u265D","\u265E","\u265F"]

print("leer" ,figuren[-leer])
print("bauer" ,figuren[-bauer])
print("springer" ,figuren[-springer])
print("laeufer" ,figuren[-laeufer])
print("turm" ,figuren[-turm])
print("dame" ,figuren[-dame])
print("koenig" ,figuren[-koenig])

def print_brett(schachbrett):
    output = ''
    bg = ["\x1b[47m", "\x1b[107m"]
    leer_farbe = ["\x1b[37m", "\x1b[97m"]
    counter = 0
    for reihe in schachbrett:
        counter += 1
        for feld in reihe:
            if feld == 0:
                output += bg[counter%2]+leer_farbe[counter%2]+figuren[feld]
            else:
                output += bg[counter%2]+"\x1b[30m"+figuren[feld]
            counter += 1
        output += '\n'
    output += "\x1b[0m"
    print(output)

print_brett(schachbrett)

[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1, -1, -1, -1, -1, -1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [4, 2, 3, 5, 6, 3, 2, 4]]
leer ♙
bauer ♟
springer ♞
laeufer ♝
turm ♜
dame ♛
koenig ♚
♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖



In [ ]:
def bewege_bauer(schachbrett, pos):
    # Checkliste: Ist da Bauer? Kann er sich bewegen?
    pos = get_index(pos)
    print(pos)
    assert abs(schachbrett[pos[0]][pos[1]])==bauer, "Hier steht kein Bauer."

    schachbrett[pos[0]][pos[1]] = leer
    schachbrett[pos[0]-1][pos[1]] = bauer

    return schachbrett

def get_index(pos):
    pos = pos.upper()
    # erhält z. B. "D4" und gibt Liste [4,3] zurück
    assert pos[0] in "ABCDEFGH" and pos[1] in "12345678", pos+" ist keine gültige Position."

    counter = 0
    for buchstabe in "ABCDEFGH":
        if buchstabe==pos[0]:
            break
        counter += 1

    liste = [8-int(pos[1]), counter]
    return liste

schachbrett = reset_schachbrett()
print_brett(schachbrett)
schachbrett = bewege_bauer(schachbrett, "B2")
print_brett(schachbrett)
schachbrett = bewege_bauer(schachbrett, "B3")
print_brett(schachbrett)
schachbrett = bewege_bauer(schachbrett, "B4")
print_brett(schachbrett)

♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖

[6, 1]
♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖

[5, 1]
♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖

[4, 1]
♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖



In [ ]:
schachbrett_fig = [[figuren[fig] for fig in lis] for lis in schachbrett]
print(*schachbrett_fig, sep="\n")

['♜', '♞', '♝', '♛', '♚', '♝', '♞', '♜']
['♟', '♟', '♟', '♟', '♟', '♟', '♟', '♟']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['♙', '♙', '♙', '♙', '♙', '♙', '♙', '♙']
['♖', '♘', '♗', '♕', '♔', '♗', '♘', '♖']


In [ ]:
!python3 --version

Python 3.10.12
